In [1]:
spark.stop()
sc.stop()

NameError: name 'spark' is not defined

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import sys
import string
from csv import reader
from functools import reduce
from pyspark.sql import functions as f
from collections import defaultdict
import datetime
%load_ext autoreload
%autoreload 2
sys.path.insert(0, '../src')
from tableCollections import TableCollections

In [3]:
spark = SparkSession \
                .builder \
                .appName("TableCollections") \
                .config("spark.io.compression.codec", "snappy") \
                .config("spark.shuffle.service.enabled", "false") \
                .config("spark.dynamicAllocation.enabled", "false") \
                .config("spark.rdd.compress", "true") \
                .getOrCreate()
sc = spark.sparkContext

In [4]:
parkingTable = spark.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/parking-violations-header.csv')
openTable = spark.read.format('csv').options(header='true',inferschema='true').load('/user/ecc290/HW1data/open-violations-header.csv')

In [16]:
tc = TableCollections(spark, sc)
tc.register(openTable, "open")
tc.register(parkingTable, "parking")
tc.numColWithinRange(0, 1000000000000).show()
tc.timeColWithinRange(datetime.datetime(1994,1,1), datetime.datetime(2018,5,1)).show()

num metadata file exists for table open
num metadata file exists for table parking
timestamp metadata file exists for table parking
+--------------+---------+
|       colName|tableName|
+--------------+---------+
|summons_number|     open|
|   fine_amount|     open|
|summons_number|  parking|
|violation_code|  parking|
+--------------+---------+

+----------+---------+
|   colName|tableName|
+----------+---------+
|issue_date|  parking|
+----------+---------+



In [17]:
tc = TableCollections(spark, sc)
dirname = '/user/jp4989/open_data_csv/'
found = False
with open('data_ids', 'r') as g:
    ids = g.readlines()
    for each in ids:
        if each.strip() == '54k3-2wtq':
            found = True
        if not found:
            continue
        filename = dirname+each.strip()+'.csv'
        df = spark.read.format('csv').options(header='true',inferschema='true').load(filename)
        if not df.rdd.isEmpty():
            tc.register(df, 'nyc_'+each.strip().replace('-','_'))

In [4]:
tc = TableCollections(spark, sc)
dirname = '/user/jp4989/open_data_csv/'
with open('data_ids', 'r') as g:
    ids = g.readlines()
    for each in ids:
        tc.add_registered_table_name('nyc_'+each.strip().replace('-','_'))
        if each.strip() == '2n4x-d97d':
            break

In [6]:
tc.numColWithinRange(0, 100).show()

+--------------------+-------------+
|             colName|    tableName|
+--------------------+-------------+
|City Council Dist...|nyc_22rf_yxcy|
| Community Districts|nyc_22rf_yxcy|
|  Borough Boundaries|nyc_22rf_yxcy|
|    Police Precincts|nyc_22rf_yxcy|
|         % Level 3+4|nyc_26kp_bgdh|
|           % Level 2|nyc_26kp_bgdh|
|           % Level 1|nyc_26kp_bgdh|
|           % Level 3|nyc_26kp_bgdh|
|  Average Class Size|nyc_276h_y36a|
|          Level3+4_%|nyc_27h8_t3wt|
|            Level2_%|nyc_27h8_t3wt|
|            Level3_%|nyc_27h8_t3wt|
|            district|nyc_27h8_t3wt|
|            Level4_%|nyc_27h8_t3wt|
|            Level1_%|nyc_27h8_t3wt|
|            latitude|nyc_29km_avyc|
|   Pct Level 3 and 4|nyc_2bh6_qmgg|
|         Pct Level 3|nyc_2bh6_qmgg|
|         Pct Level 4|nyc_2bh6_qmgg|
|         Pct Level 2|nyc_2bh6_qmgg|
+--------------------+-------------+
only showing top 20 rows



In [7]:
# return NYC longitude columns
tc.numColWithinRange(-76, -70).show()

+---------+-------------+
|  colName|    tableName|
+---------+-------------+
|longitude|nyc_29km_avyc|
|Longitude|nyc_2fra_mtpn|
|Longitude|nyc_2n4x_d97d|
|Longitude|nyc_2q48_ip9a|
|  Borough|nyc_35f6_8qd2|
|Longitude|nyc_35sw_rdxj|
|Longitude|nyc_36hn_wea6|
|Longitude|nyc_37fm_7uaa|
|Longitude|nyc_37it_gmcp|
|Longitude|nyc_3qfc_4tta|
|Longitude|nyc_3rfa_3xsf|
|Longitude|nyc_3spy_rjpw|
|Longitude|nyc_48pb_zy2g|
|Longitude|nyc_4wf2_7kdu|
|Longitude|nyc_4zdr_zwdi|
|Longitude|nyc_56u5_n9sa|
|Longitude|nyc_57mv_nv28|
|Longitude|nyc_59kj_x8nc|
|Longitude|nyc_59t5_r7nb|
|Longitude|nyc_5e24_x4wa|
+---------+-------------+
only showing top 20 rows



In [8]:
# return NYC latitude columns
tc.numColWithinRange(39, 43).show()

+--------+-------------+
| colName|    tableName|
+--------+-------------+
|latitude|nyc_29km_avyc|
|Latitude|nyc_2fra_mtpn|
|Latitude|nyc_2n4x_d97d|
|Latitude|nyc_2q48_ip9a|
|Latitude|nyc_35sw_rdxj|
|Latitude|nyc_36hn_wea6|
|Latitude|nyc_37fm_7uaa|
|Latitude|nyc_37it_gmcp|
|Latitude|nyc_3qfc_4tta|
|Latitude|nyc_3rfa_3xsf|
|Latitude|nyc_3spy_rjpw|
|Latitude|nyc_48pb_zy2g|
|Latitude|nyc_4wf2_7kdu|
|Latitude|nyc_4zdr_zwdi|
|Latitude|nyc_56u5_n9sa|
|Latitude|nyc_57mv_nv28|
|Latitude|nyc_59kj_x8nc|
|Latitude|nyc_59t5_r7nb|
|Latitude|nyc_5e24_x4wa|
|Latitude|nyc_5fn4_dr26|
+--------+-------------+
only showing top 20 rows



In [9]:
# return columns between 0 and 1
tc.numColWithinRange(0, 1).show()

+-------+---------+
|colName|tableName|
+-------+---------+
+-------+---------+



In [10]:
# return columns between 0 and 1
tc.numColWithinRange(-0.1, 1.1).show()

+--------------------+-------------+
|             colName|    tableName|
+--------------------+-------------+
|Unregistered Vehi...|nyc_2bnn_yakx|
|COUNT PUBLIC ASSI...|nyc_2t32_hbca|
|COUNT CITIZEN STA...|nyc_2t32_hbca|
|PERCENT GENDER UN...|nyc_2t32_hbca|
|COUNT PACIFIC ISL...|nyc_2t32_hbca|
|COUNT AMERICAN IN...|nyc_2t32_hbca|
|COUNT GENDER UNKNOWN|nyc_2t32_hbca|
|        Derived View|nyc_56b7_s9t5|
|Average Minors pe...|nyc_5r5y_pvs3|
|            Latitude|nyc_656a_faqy|
|PERCENT NRECEIVES...|nyc_6bic_qvek|
|PERCENT RECEIVES ...|nyc_6bic_qvek|
|PERCENT PUBLIC AS...|nyc_6bic_qvek|
|PERCENT PERMANENT...|nyc_6bic_qvek|
|COUNT PUBLIC ASSI...|nyc_6bic_qvek|
|PERCENT CITIZEN S...|nyc_6bic_qvek|
|PERCENT OTHER CIT...|nyc_6bic_qvek|
|COUNT CITIZEN STA...|nyc_6bic_qvek|
|PERCENT WHITE NON...|nyc_6bic_qvek|
|PERCENT ASIAN NON...|nyc_6bic_qvek|
+--------------------+-------------+
only showing top 20 rows



In [32]:
tc.getNumRange(["nyc_2n4x_d97d^Latitude", "nyc_29km_avyc^latitude", "nyc_35f6_8qd2^Borough"]).show()

+-------------+--------+----------+----------+
|    tableName| colName|       min|       max|
+-------------+--------+----------+----------+
|nyc_2n4x_d97d|Latitude|  40.76135| 40.769233|
|nyc_29km_avyc|latitude| 40.508283| 40.906225|
|nyc_35f6_8qd2| Borough|-74.124757|-74.022721|
+-------------+--------+----------+----------+



In [31]:
tc.getTimeRange(["nyc_6246_94tp^TCO Obtained Date",  "nyc_avz8_mqzz^DropOff_datetime", "nyc_cwx7_agsh^StartDate"]).show()

+-------------+-----------------+-------------------+-------------------+
|    tableName|          colName|                min|                max|
+-------------+-----------------+-------------------+-------------------+
|nyc_6246_94tp|TCO Obtained Date|2004-01-08 00:00:00|2018-03-14 00:00:00|
|nyc_avz8_mqzz| DropOff_datetime|2017-01-02 13:28:00|2018-12-31 00:20:00|
|nyc_cwx7_agsh|        StartDate|2006-04-03 00:00:00|2017-06-01 00:00:00|
+-------------+-----------------+-------------------+-------------------+



In [37]:
tc.getSimilarNumCols("a^b")

LookupError: Given column does not exist!

In [6]:
from pyspark.sql import Row
from csv import reader
from functools import reduce
from pyspark.sql import functions as f
from collections import defaultdict
import datetime
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, TimestampType, StringType

schema = StructType([
            StructField("colName", StringType(), True),
            StructField("min", DoubleType(), True),
            StructField("max", DoubleType(), True)])
currentTable = spark.read.csv("nyc_d8cz_kh5x_num_metadata.csv",header=False,schema=schema, sep='^')
minMax = currentTable.where(currentTable.colName == "2008")

In [45]:
a = minMax.collect()

In [46]:
a[0][0]

'2008'

In [48]:
currentTable.show()

+-------+---+-----+
|colName|min|  max|
+-------+---+-----+
|  Total|5.0|554.0|
|   2006|0.0|136.0|
|   2009|0.0|110.0|
|   2005|0.0|100.0|
|   2007|0.0|118.0|
|   2008|0.0| 90.0|
+-------+---+-----+



In [53]:
currentTable.select(currentTable.max-currentTable.min).show()

+-----------+
|(max - min)|
+-----------+
|      549.0|
|      136.0|
|      110.0|
|      100.0|
|      118.0|
|       90.0|
+-----------+



In [51]:
currentTable.select(min(currentTable.max, 10)-max(currentTable.min,3) ).show()

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [66]:
# 3, 7
rows = currentTable.rdd.collect()
for each in rows:
    intersection = min(each["max"], 7) - max(each["min"], 3)
    union = max(each["max"], 7) - min(each["min"], 3)
    print(each["colName"], intersection/union)

Total 0.003629764065335753
2006 0.029411764705882353
2009 0.03636363636363636
2005 0.04
2007 0.03389830508474576
2008 0.044444444444444446


In [12]:
#start: 4:05pm
#end: ?
print(datetime.datetime.now())
tc.getSimilarNumCols("nyc_2n4x_d97d^Latitude").show() 
print(datetime.datetime.now())

2018-05-02 17:09:53.071277


KeyboardInterrupt: 

In [19]:
#30 seconds with or without sort, 19 seconds if ignoring no intersection
print(datetime.datetime.now())
tc.getSimilarNumCols("nyc_2n4x_d97d^Latitude", 0.01).show() 

2018-05-02 17:17:53.960565
+--------+-------------+--------------------+
| colName|    tableName|                 iou|
+--------+-------------+--------------------+
|Latitude|nyc_2n4x_d97d|                 1.0|
|latitude|nyc_29km_avyc|0.019809419463136916|
|Latitude|nyc_2fra_mtpn|0.019043092869228086|
+--------+-------------+--------------------+



In [22]:
tc.getSimilarNumCols("nyc_2n4x_d97d^Latitude", 1.1).show() 

+---------+-------+---+
|tableName|colName|iou|
+---------+-------+---+
+---------+-------+---+



In [14]:
print(datetime.datetime.now())

2018-05-02 17:12:03.638909


In [90]:
minMax = currentTable.where(currentTable.colName == "2008").collect()[0]
currentMin = minMax["min"]
currentMax = minMax["max"]

In [91]:
currentMin

0.0

In [93]:
currentMax

90.0

In [95]:
currentTable.where(currentTable.min <= currentMin).show()

+-------+---+-----+
|colName|min|  max|
+-------+---+-----+
|   2006|0.0|136.0|
|   2009|0.0|110.0|
|   2005|0.0|100.0|
|   2007|0.0|118.0|
|   2008|0.0| 90.0|
+-------+---+-----+



In [97]:
a = currentTable.where(currentTable.min <= currentMin)
b = currentTable.where(currentTable.min > currentMin)
c = currentTable.where(currentTable.max <= currentMax)
d = currentTable.where(currentTable.max > currentMax)

In [98]:
a.show()

+-------+---+-----+
|colName|min|  max|
+-------+---+-----+
|   2006|0.0|136.0|
|   2009|0.0|110.0|
|   2005|0.0|100.0|
|   2007|0.0|118.0|
|   2008|0.0| 90.0|
+-------+---+-----+



In [99]:
b.show()

+-------+---+-----+
|colName|min|  max|
+-------+---+-----+
|  Total|5.0|554.0|
+-------+---+-----+



In [100]:
c.show()

+-------+---+----+
|colName|min| max|
+-------+---+----+
|   2008|0.0|90.0|
+-------+---+----+



In [101]:
d.show()

+-------+---+-----+
|colName|min|  max|
+-------+---+-----+
|  Total|5.0|554.0|
|   2006|0.0|136.0|
|   2009|0.0|110.0|
|   2005|0.0|100.0|
|   2007|0.0|118.0|
+-------+---+-----+



In [114]:
#currentTable.min <= currentMin
#currentTable.max <= currentMax
intersection = currentTable.max - currentMin
union = currentMax - currentTable.min
case_1 = a.intersect(c)
case_1.select(case_1.colName,(case_1.max-currentMin)/(currentMax-case_1.min)).show()

+-------+----------------------------+
|colName|((max - 0.0) / (90.0 - min))|
+-------+----------------------------+
|   2008|                         1.0|
+-------+----------------------------+



In [112]:
#currentTable.min <= currentMin
#currentTable.max > currentMax
intersection = currentMax - currentMin
union = currentTable.max - currentTable.min
case_2 = a.intersect(d)
case_2.select(case_2.colName,(currentMax - currentMin)/(case_2.max-case_2.min)).show()

+-------+--------------------+
|colName|(90.0 / (max - min))|
+-------+--------------------+
|   2005|                 0.9|
|   2007|  0.7627118644067796|
|   2009|  0.8181818181818182|
|   2006|  0.6617647058823529|
+-------+--------------------+



In [115]:
#currentTable.min > currentMin
#currentTable.max <= currentMax
intersection = currentTable.max - currentTable.min
union = currentMax - currentMin
case_3 = b.intersect(c)
case_3.select(case_3.colName,(case_3.max-case_3.min)/(currentMax-currentMin)).show()

+-------+--------------------+
|colName|((max - min) / 90.0)|
+-------+--------------------+
+-------+--------------------+



In [119]:
#currentTable.min > currentMin
#currentTable.max > currentMax
intersection = currentMax - currentTable.min
union = currentTable.max - currentMin
case_4 = b.intersect(d)
case_4.select(case_4.colName,((currentMax - case_4.min)/(case_4.max-currentMin)).alias("iou")).show()

+-------+-------------------+
|colName|                iou|
+-------+-------------------+
|  Total|0.15342960288808663|
+-------+-------------------+



In [11]:
currentTable.rdd.isEmpty()

False

In [18]:
spark.createDataFrame(sc.emptyRDD(), schema)

DataFrame[colName: string, min: double, max: double]